In [1]:
import numpy as np
import pandas as pd
np.random.seed(7)

In [2]:
def toNumber(inputList):
    outList = []    
    for word in inputList:
        newWord = []
        for char in word:
            newWord.append(ord(char))
        outList.append(newWord)          
    return outList

def fitToSize(word, n):
    wordFit = word[:n]
    if n > len(word):
        for i in range(n - len(word)):
            wordFit.append(0)    
    return wordFit

def fixSymbolEncode(x):
    
    if x == 8217:
        return 39
    elif x == 8220:
        return 34
    elif x == 8221:
        return 34
    elif x == 8230:
        return 46
    elif x == 8722:
        return 45
    elif x == 8211:
        return 45
    elif x == 8216:
        return 39
    elif x == 8226:
        return 46
    elif x == 8364:
        return 36
    return x

In [3]:
data = pd.read_csv('dates.csv')
Y = np.array(data.word)
X = np.array(data.label)

In [4]:
for i in range(len(X)):
    if type(X[i]) != str:
        X[i] = ''
for x in X:
    if type(x) != str:        
        print(x)

In [5]:
data = toNumber(X)

In [6]:
for i in range(len(data)):
    data[i] = fitToSize(data[i],8)

In [7]:
for i in range(len(data)):
    for j in range(len(data[i])):        
            data[i][j] = fixSymbolEncode(data[i][j])

In [8]:
maxl = 0;
maxa = []
for i in range(len(data)):
    for j in range(len(data[i])):        
           if data[i][j] > 126:
                maxl+=1                
print(maxl)

3042


In [7]:
labels = []
for y in Y:
    ans = [0,0,0]
    if y == 'NONE':
        ans[0] = 1        
    elif y == 'D_ST': 
        ans[1] = 1   
    elif y == 'D_PT':
        ans[2] = 1   
    labels.append(ans)

In [8]:
from keras.layers import Input
from keras.layers import TimeDistributed
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dense
from keras.models import Model


Using TensorFlow backend.


In [36]:
inputLayer = Input(dtype='int32', shape=(None,8,), name = 'inputLayer')
embeddingLayer = TimeDistributed(Embedding(input_dim = 9000, output_dim = 10, trainable=True, 
                                         name = 'embeddingLayer'), name='TDEmbedding')(inputLayer)
lstmLayer = TimeDistributed(LSTM(50, return_sequences=False, consume_less='gpu', 
                                name = 'lstmLayer'), name='TDlstm')(embeddingLayer)
outputLayer = TimeDistributed(Dense(output_dim=3,init='zero',activation='softmax',name='outputLayer'),
                              name='TDout')(lstmLayer)
model = Model(input=inputLayer, output=outputLayer)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy','fmeasure'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
inputLayer (InputLayer)          (None, None, 8)       0                                            
____________________________________________________________________________________________________
TDEmbedding (TimeDistributed)    (None, None, 8, 10)   90000       inputLayer[0][0]                 
____________________________________________________________________________________________________
TDlstm (TimeDistributed)         (None, None, 50)      12200       TDEmbedding[0][0]                
____________________________________________________________________________________________________
TDout (TimeDistributed)          (None, None, 3)       153         TDlstm[0][0]                     
Total params: 102,353
Trainable params: 102,353
Non-trainable params: 0
___________________

In [10]:
data = np.array(data)
data.shape

(459952, 8)

In [11]:
data.shape

(459952, 8)

In [12]:
trainX = data[:459950]
trainX = trainX.reshape(45995,10,8)
trainX.shape

(45995, 10, 8)

In [13]:
trainX[:2]

array([[[ 82, 117, 115, 115, 105,  97,  39, 115],
        [ 76, 101, 110, 101, 110, 101, 114, 103],
        [105, 110,   0,   0,   0,   0,   0,   0],
        [ 99, 114, 105, 115, 105, 115,   0,   0],
        [100, 117, 101,   0,   0,   0,   0,   0],
        [116, 111,   0,   0,   0,   0,   0,   0],
        [109,  97, 110,  97, 103, 101, 109, 101],
        [105, 110, 101, 102, 102, 105,  99, 105],
        [ 77,  79,  83,  67,  79,  87,   0,   0],
        [ 77,  97, 114,   0,   0,   0,   0,   0]],

       [[ 49,  57,   0,   0,   0,   0,   0,   0],
        [ 40,  80,  82,  73,  77,  69,  41,   0],
        [ 45,  45,   0,   0,   0,   0,   0,   0],
        [ 82, 117, 115, 115, 105,  97, 110,   0],
        [112, 111, 119, 101, 114,   0,   0,   0],
        [103, 114, 105, 100,   0,   0,   0,   0],
        [ 99, 111, 109, 112,  97, 110, 121,   0],
        [ 76, 101, 110, 101, 110, 101, 114, 103],
        [ 97,   0,   0,   0,   0,   0,   0,   0],
        [117, 110, 105, 116,   0,   0,   0,   0]

In [14]:
trainY = np.array(labels[:459950])
trainY = trainY.reshape(45995,10,3)
trainY.shape

(45995, 10, 3)

In [15]:
trainY[:2]

array([[[1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [0, 1, 0]],

       [[0, 0, 1],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0]]])

In [ ]:
model.fit(trainX[:5000], trainY[:5000])

Epoch 1/10
5000/5000 [==============================] - 4s - loss: 0.1843 - acc: 0.9774 - fmeasure: 0.8948          

In [ ]:
model.train_on_batch(trainX[:2000], trainY[:2000])

In [27]:
model.train_on_batch(trainX[2000:4000], trainY[2000:4000])

[0.0059518148, 0.99049997, 0.9904747]

In [28]:
model.train_on_batch(trainX[4000:6000], trainY[4000:6000])

[0.0046998467, 0.99264997, 0.99267405]

In [38]:
answer = model.predict(trainX[0:20])

In [30]:
trainY[0:20]

array([[[1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [0, 1, 0]],

       [[0, 0, 1],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0]],

       [[1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0]],

       [[1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0]],

       [[1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0]],

       [[1, 0, 0],
        [1, 0, 0],
  

In [39]:
answer[0:20]

array([[[  9.99283254e-01,   4.97508561e-04,   2.19320136e-04],
        [  9.98878658e-01,   8.33900122e-04,   2.87417206e-04],
        [  9.98675525e-01,   9.84787242e-04,   3.39614548e-04],
        [  9.99032021e-01,   7.18165888e-04,   2.49880308e-04],
        [  9.98542309e-01,   1.07514206e-03,   3.82595666e-04],
        [  9.97979939e-01,   1.46344001e-03,   5.56604529e-04],
        [  9.98250902e-01,   1.28949946e-03,   4.59560077e-04],
        [  9.99218941e-01,   5.49926364e-04,   2.31073354e-04],
        [  9.90443766e-01,   6.55981945e-03,   2.99632805e-03],
        [  9.47459161e-01,   2.57539880e-02,   2.67868843e-02]],

       [[  8.52728367e-01,   7.49716386e-02,   7.23000243e-02],
        [  9.99236345e-01,   5.51836216e-04,   2.11846171e-04],
        [  9.99174178e-01,   6.11724332e-04,   2.14035274e-04],
        [  9.99149799e-01,   6.28301001e-04,   2.21916402e-04],
        [  9.98976827e-01,   7.67141290e-04,   2.56067229e-04],
        [  9.98726547e-01,   9.5789681

In [43]:
answer.shape

(20, 10, 3)

In [44]:
a = answer.reshape(answer.shape[0]*answer.shape[1], answer.shape[2])

In [45]:
a.shape


(200, 3)

In [46]:
np.argmax(a[0])

0

In [47]:
og = []
for i in range(a.shape[0]):
    if np.argmax(a[i]) == 0:
        og.append('NONE')
    elif np.argmax(a[i]) == 1:
        og.append('D_ST')
    elif np.argmax(a[i]) == 2:
        og.append('D_PT')        

In [48]:
og

['NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 

In [18]:
testdf = pd.DataFrame(answer, labels)
testdf.to_csv('testdf.csv')